In [1]:
from Physics import *
from Tools import *
import numpy as np

In [2]:
class DynamicalSimulator(StaticSimulator):
    def __init__(self,Particles,ForceFieldGenerator):
        super().__init__(Particles,ForceFieldGenerator)
        self.Scale  = 1.0
        self.Factor = 10.
        self.Frozen = []
        
    def SimulateEuler(self,Time,Steps):
        t = 0
        dt = Time/Steps
        
        counter = 0
        while t < Time: 
            counter += 1
            t += dt
            self.forceField.Evolve(dt,"Euler")

            for Particle in self.Particles:
                Particle.Acc = self.forceField(Particle.Pos)
                Particle.Evolve(dt)
            
            if counter%100 != 0:
                continue
            
            ToRemove = []
            for i,particle in enumerate(self.Particles):
                if(np.sqrt(particle.Pos*particle.Pos)) > self.Scale*self.Factor:
                    #print("Warning Particle",particle.Nam,"is out of Scope")
                    #print("Freezing it")
                    ToRemove.append(i)
            ToRemove.sort(key=lambda x : -x)
            N = len(self.Particles)
            for i in range(len(ToRemove)):
                    self.Frozen.append(self.Particles[ToRemove[i]])
                    del self.Particles[ToRemove[i]]

In [3]:
Sun1 = Particle("Sun 1",6.0,Vector(5.0/6.0,0,0), Vector(0,np.pi,0),RED)
Sun2 = Particle("Sun 2",2.0,Vector(-15.0/6.0,0,0),Vector(0,-3*np.pi,0),BLUE)

BinaryStarPotential  = ForceFieldGenerator([Sun1,Sun2])
    
Particles = []
for i in range(2):
    Position = Vector(Random(1,2),0,0)
    Velocity = Vector(0,Random(-12,-10),0)
    Mass = 1E-6   # Irrelevant unless interactions are turned-on        
    Particles.append(Particle("Planet "+str(i+1),Mass,Position,Velocity,WHITE))

In [4]:
Test_Static = True
Test_Vanilla = False
Test_OrbitFinder = False

Time  = 1
Steps = 2000

BinaryStarPotential = ForceFieldGenerator([Sun1,Sun2])

if Test_Static:

    BinaryStarSimulation = StaticSimulator(Particles,BinaryStarPotential)
    BinaryStarSimulation.ClearTrajectories()
    BinaryStarSimulation.GoToCM()
    BinaryStarSimulation.Simulate(Time,Steps,"Euler")
    BinaryStarSimulation.GeneratePlots({
        "mode"    :"Save",
        "filename":"Binary Star Simulation"})
    #BinaryStarSimulation.GenerateVideo("Binary System 2")
    
if Test_Vanilla:

    BasicSimulator = Simulator([Sun1,Sun2])
    BasicSimulator.ClearTrajectories()
    BasicSimulator.GoToCM()
    BasicSimulator.Simulate(Time,Steps)
    BasicSimulator.GeneratePlots(mode="Show")
    #BasicSimulator.GenerateVideo("Binary System 1")

if Test_OrbitFinder:

    OF = OrbitFinder([],BinaryStarPotential)
    OF.GoToCM()
    OF.FindScale(5,5E-3)

TypeError: GeneratePlots() got an unexpected keyword argument 'mode'

In [13]:
# Here we perform the data collection on the Designed Binary System
# Search Parameters 

BinaryStarPotential = ForceFieldGenerator([Sun1,Sun2])

MinR = 1
MaxR = 25
RSteps = 10
ThetaSteps = 30

Distances = [ MinR + (MaxR-MinR)*i/RSteps for i in range(RSteps+1) ]
Angles    = [ -np.pi + 2*np.pi*i/ThetaSteps for i in range(ThetaSteps+1) ]

M = 0
f = open("Saved Particles.py","w")
for particle in BinaryStarPotential.Particles:
    M += particle.Mas
f.write("Particles = [")
for R in Distances:
    # Here we sample the ring of radius = distance
    Planets = []
    # The speed will be computed according to Kepler's Law
    # v^2 = G(MSuns)/R^3
    T = np.sqrt(4*np.pi**2*R**3/(G*M))
    V = 2*np.pi*R/T
    for i,angle in enumerate(Angles):
        Velocity = Vector(-V*np.sin(angle),V*np.cos(angle),0)
        Position = Vector(R*np.cos(angle),R*np.sin(angle),0)
        Mass = 1E-3
        Part = Particle("Planet "+str(i+1),Mass,Position,Velocity,WHITE)
        Planets.append(Part)
    BinaryStarSimulation = DynamicalSimulator(Planets,BinaryStarPotential)
    BinaryStarSimulation.Scale = R
    BinaryStarSimulation.Factor = 20
    Time = 5*T
    Steps = int(500)
    print("Starting with R =",R)
    print("Going for",Time,"years with",Steps,"steps over time")
    BinaryStarSimulation.ClearTrajectories()
    BinaryStarSimulation.GoToCM()
    BinaryStarSimulation.Simulate(Time,Steps,"Euler")
    BinaryStarSimulation.GeneratePlots({
        "mode"    :"Save",
        "filename":"Simulation_R_"+str(R)})
    print("Done with R =",R)
    f.write('\n\t{\n\t\t"R":'+str(R)+',\n\t\t"Particles":[\n')
    for i,particle in enumerate(BinaryStarSimulation.Particles):
        line = '\t\t\t{"Position":Vector'+str(particle.IniPos)+',"Velocity":Vector'+str(particle.IniVel)+"}"
        if(i != len(BinaryStarSimulation.Particles)-1):
            line += ","
        line += "\n"
        f.write(line)
    f.write("\t\t]\n\t}"+("" if R==MaxR else ","))
    f.flush()
f.write("\n]\n")

f.close()

Starting with R = 1.0
Going for 1.7677669529663689 years with 500 steps over time
Done with R = 1.0
Starting with R = 3.4
Going for 11.082644088844502 years with 500 steps over time
Done with R = 3.4
Starting with R = 5.8
Going for 24.69261023059328 years with 500 steps over time
Done with R = 5.8
Starting with R = 8.2
Going for 41.50933629919899 years with 500 steps over time
Done with R = 8.2
Starting with R = 10.6
Going for 61.00758149607309 years with 500 steps over time
Done with R = 10.6
Starting with R = 13.0
Going for 82.85906709588275 years with 500 steps over time
Done with R = 13.0
Starting with R = 15.4
Going for 106.83316432643937 years with 500 steps over time
Done with R = 15.4
Starting with R = 17.8
Going for 132.75626162256907 years with 500 steps over time
Done with R = 17.8
Starting with R = 20.2
Going for 160.4915106789141 years with 500 steps over time
Done with R = 20.2
Starting with R = 22.6
Going for 189.9274203478792 years with 500 steps over time
Done with R =